In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [6]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 7
ROUNDS = 7
OTHER_DATE = '2021-11-11'
_RESETS = False
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Searching for ibm_sherbrooke and 23.10.27_07h46_300pts_2std


# Get IQ data

In [7]:
SHOTS = int(1e4)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]
# NOISE_LIST = None
P_AMBIG = 0.3

IQ_data = simulator.generate_IQ(SHOTS, noise_list=NOISE_LIST)
IQ_data_extreme = simulator.generate_extreme_IQ(SHOTS, P_AMBIG, noise_list=NOISE_LIST)

100%|██████████| 8871/8871 [00:03<00:00, 2765.61it/s]


# Score IQ data

In [20]:
from soft_info import get_repcode_IQ_map
print(IQ_data.shape)
IQ_map = get_repcode_IQ_map(simulator.layout, ROUNDS)
simulator.kde_dict
# simulator.scaler_dict
# print(IQ_map)

(10000, 49)


{0: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 1: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 2: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 3: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 4: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 5: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 6: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 7: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 8: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 9: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 10: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 11: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 12: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 13: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 14: [KernelDensity(bandwidth=0.2), KernelDensity(bandwidth=0.2)],
 15: 

In [22]:
import numpy as np
from tqdm import tqdm

# Assuming IQ_data is an array of complex numbers of shape (10000, 49)

# Initialize the results array
results = np.zeros((IQ_data.shape[0], IQ_data.shape[1], 2))  # Shape (10000, 49, 2)

# Convert complex IQ data to 2D (real and imaginary parts)
IQ_data_2D = np.stack((IQ_data.real, IQ_data.imag), axis=-1)  # Shape (10000, 49, 2)

# Group IQ points by qubit
qubit_IQ_points = {qubit: [] for qubit in set(IQ_map.values())}
for col in range(IQ_data_2D.shape[1]):
    qubit = IQ_map[col]
    qubit_IQ_points[qubit].append(IQ_data_2D[:, col, :])  # Append 2D data

# Process each qubit group
for qubit, iq_points in tqdm(qubit_IQ_points.items()):
    iq_points = np.concatenate(iq_points, axis=0)  # Shape (num_samples, 2) for 2D data

    # Rescale the IQ points
    scaler = simulator.scaler_dict[qubit]
    rescaled_IQ_points = scaler.transform(iq_points)

    # Score the probability using KDEs
    kde0 = simulator.kde_dict[qubit][0]
    kde1 = simulator.kde_dict[qubit][1]

    score0 = kde0.score_samples(rescaled_IQ_points)
    score1 = kde1.score_samples(rescaled_IQ_points)

    # Distribute the scores back to the original structure
    for i in range(IQ_data.shape[0]):
        for col_index, col in enumerate(IQ_map):
            if IQ_map[col] == qubit:
                results[i, col_index, 0] = score0[i]
                results[i, col_index, 1] = score1[i]

# 'results' now contains the KDE scores for each IQ point


100%|██████████| 13/13 [04:21<00:00, 20.13s/it]
